In [ ]:
"""
Fine-grained classification practice with Flower-17
"""

# Python Packages
import argparse
import os
import time
# 3rd Party Packages
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
import cv2
# User Packages
from start.preprocessing import ImageToTensorPreprocessor, ResizePreprocessor, ColorSpacePreprocessor
from start.loader import ImageDataset
from start.model import MiniVGGNet


In [ ]:
# Load Flowers-17 dataset
dataset = ImageDataset(
    preprocessors=[
        ResizePreprocessor(224, 224, aspect_preserving=True),
        ColorSpacePreprocessor(conversion=cv2.COLOR_BGR2GRAY),
        ImageToTensorPreprocessor()
    ]
)
(data, labels) = dataset.load(
    dataset_path=r'/home/share/dataset/flowers17',
    verbosity=80
)

print('data shape: {}'.format(data.shape))
print('labels shape: {}'.format(labels.shape))

classes = set(labels)

# Normalize data
data = data.astype(np.float) / 255.0

In [ ]:
# Setup data splits
# Partition into train and test splits
(trainX, testX, trainY, testY) = train_test_split(
    data, labels,
    test_size=0.2,
    random_state=int(time.time()),
    stratify=list(labels)
)
(valX, testX, valY, testY) = train_test_split(
    testX, testY,
    test_size=0.4,
    random_state=int(time.time()),
    stratify=list(testY)
)

# Binarize output to one hot vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
valY = lb.fit_transform(valY)
testY = lb.fit_transform(testY)

# Data augmentation
augmenter = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Initialize the optimizer and model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
import pycm
from tensorflow.keras.models import save_model
import time

BATCH_SIZE = 32
# Initialize optimizer
N_EPOCHS = 100
learning_rate = 0.01
decay_rate = learning_rate / N_EPOCHS
N_TRAINABLE_LAYERS = 152
timestamp = time.time()
OUTPUT_DIR = r'/home/share/education/deep_learning/pyimagesearch/models/flower-17-bw/'

for trainable_layer in list(range(-N_TRAINABLE_LAYERS, 0))[::-1]:
    clear_session()
    print('[INFO] compiling model to be trainable until layer {}...'.format(N_TRAINABLE_LAYERS+trainable_layer+1))
    properties = {
        'width':    224,
        'height':   224,
        'channels': 1,
        'classes':  len(classes)
    }
    #model = MiniVGGNet.build(properties)
    
    mnv2_imagenet = MobileNetV2(
        input_shape=(224, 224, 3),
        alpha=1.4,
        depth_multiplier=1, 
        include_top=False, 
        weights='imagenet', 
        input_tensor=None, 
        pooling='avg'
    )
    mnv2 = MobileNetV2(
        input_shape=(224, 224, 1),
        alpha=1.4,
        depth_multiplier=1, 
        include_top=False, 
        weights=None, 
        input_tensor=None, 
        pooling='avg'
    )
    
    for i, layer in enumerate(mnv2_imagenet.layers[3:152]):
        weights = mnv2_imagenet.get_layer(name=layer.get_config()['name']).get_weights()
        mnv2.get_layer(name=layer.get_config()['name']).set_weights(weights)
    
    layer = mnv2_imagenet.layers[2]
    weights = mnv2_imagenet.get_layer(name=layer.get_config()['name']).get_weights()
    weights[0] = np.reshape(np.mean(weights[0], axis=2), (3, 3, 1, 48))
    mnv2.get_layer(name=layer.get_config()['name']).set_weights(weights)
    
    for layer in mnv2.layers[:trainable_layer]:
        layer.trainable = False
    trainable_layer_name = mnv2.layers[trainable_layer].get_config()['name']
            
    
    model = Sequential()
    model.add(mnv2)
    # MobileNetV2 uses a GAP layer which essentially "Flattens" the feature maps already, so no call to Flatten here
    model.add(Dense(
        units=256,
        activation='relu',
        use_bias=True,
        kernel_initializer='glorot_uniform',
        bias_initializer='glorot_uniform',
        kernel_regularizer=l2(0.001),
        bias_regularizer=l2(0.001),
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None
    ))
    model.add(Dropout(0.4))
    model.add(Dense(
        units=len(classes),
        activation='softmax',
        use_bias=True,
        kernel_initializer='glorot_uniform',
        bias_initializer='glorot_uniform',
        kernel_regularizer=l2(0.001),
        bias_regularizer=l2(0.001),
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None
    ))
    
    """
    tb_callback = TensorBoard(
        log_dir='./logs/{}'.format(timestamp), 
        histogram_freq=2, 
        batch_size=BATCH_SIZE, 
        write_graph=False, 
        write_grads=False, 
        write_images=False, 
        embeddings_freq=0,
        embeddings_layer_names=None, 
        embeddings_metadata=None, 
        embeddings_data=None
    )
    """
    
    opt = SGD(
        lr=learning_rate,
        momentum=0,
        decay=0,
        nesterov=False
    )
    """
    opt = Adam(
        lr=learning_rate,
        beta_1=0.99,
        beta_2=0.999,
        epsilon=0.1,
        decay=decay_rate,
        amsgrad=False
    )
    """
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )
    
    # Train the network
    print('[INFO] training network...')
    history = model.fit_generator(
        augmenter.flow(trainX, trainY, 
                       batch_size=BATCH_SIZE),
        validation_data=(valX, valY),
        steps_per_epoch=len(trainX) // BATCH_SIZE,
        epochs=N_EPOCHS,
        callbacks=[],
        verbose=1
    )

    # Evaluate the network
    print('[INFO] evaluating network...')
    predictions = model.predict(testX, batch_size=BATCH_SIZE)
    
    cm = pycm.ConfusionMatrix(
        actual_vector=lb.inverse_transform(testY),
        predict_vector=lb.inverse_transform(predictions)
    )
    cm.save_html(os.path.join(OUTPUT_DIR, '{}_confusion_matrix_layer_{}.html'.format(timestamp, trainable_layer_name)))
    
    model_string = ('{0}_flowers-17-bw-valacc{1:.3f}-valloss{2:.3f}_layer_{3}'.format(
        timestamp, 
        history.history['val_acc'][-1], history.history['val_loss'][-1],
        trainable_layer_name
    )).replace('.', ',')
    # Save model
    save_model(
        model,
        os.path.join(OUTPUT_DIR, model_string + '.h5')
    )
    
    # Plot the training loss and accuracy
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, N_EPOCHS), history.history['loss'], label='train_loss')
    plt.plot(np.arange(0, N_EPOCHS), history.history['val_loss'], label='val_loss')
    plt.plot(np.arange(0, N_EPOCHS), history.history['acc'], label='train_acc')
    plt.plot(np.arange(0, N_EPOCHS), history.history['val_acc'], label='val_acc')
    plt.title('Training Loss and Accuracy')
    plt.xlabel('Epoch #')
    plt.ylabel('Loss/Accuracy')
    plt.legend()
    plt.savefig(os.path.join(OUTPUT_DIR, model_string+'.png'))

## Here the most common ways to prevent overfitting in neural networks:

* Get more training data.
* Reduce the capacity of the network.
* Add weight regularization.
* Add dropout.